In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


**IMPORTING LIBRARIES**

In [2]:
import torch
import io
from torch import nn, optim
import sentencepiece as spm
from datasets import Dataset
import pandas as pd
from transformers import Trainer, TrainingArguments, XLMRobertaModel, XLMRobertaConfig,XLMRobertaForSequenceClassification
import ast
from torch.utils.data import  DataLoader
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import shutil
import re
from sklearn.metrics import accuracy_score
from transformers import EvalPrediction
from gensim.models import KeyedVectors
from torch.nn.utils.rnn import pad_sequence
from transformers import XLMRobertaForTokenClassification
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'


**IMPORTING DATA**

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
file_path = '/content/drive/MyDrive/Error_d/data.csv'

In [5]:
df = pd.read_csv(file_path)


**Preparing data**



In [6]:
columns_to_drop_without_nationality = ['USER_NUMBER', 'file_name','begin','end','Gender','wrong_word','process','main_category','sub_category_1','sub_category_2','Training Center','Nationality','nationality_encoded','correct_word','word_tokens_correct','correct_sentence','wrong_sentence']
columns_to_drop_with_nationality =  ['USER_NUMBER', 'begin','end','Gender','process','main_category','sub_category_1','sub_category_2','Training Center','Nationality','correct_word','word_tokens_correct','correct_sentence','wrong_sentence']

Training_df_without_nationality = df.drop(columns=columns_to_drop_without_nationality)
# Training_df_with_nationality =df.drop(columns=columns_to_drop_with_nationality)

In [7]:
Training_df_without_nationality['word_tokens_wrong'] = Training_df_without_nationality['word_tokens_wrong'].apply(lambda x: ast.literal_eval(x))

In [8]:
def clean_token(token, symbols):
    if isinstance(token, str):  # Ensure token is a string
        # Remove unwanted symbols
        for symbol in symbols:
            token = token.replace(symbol, '')
        # Remove digits (if needed)
        token = re.sub(r'\d+', '', token)
    return token

In [9]:
def clean_list(token_list, symbols):
    return [clean_token(token, symbols) for token in token_list]

In [10]:
symbols_to_remove = [
    '@', '#', '$', '%', '^', '&', '*', '(', ')', '[', ']', '{', '}', '"',
    '*\r', '+', ',', 'ø', '>>', 'CV', 'EXPO', 'None', 'alfare', 'fat', '«',
    '\\u200', '\\u200c', '\\u200f', '\\u200e', '\r','\u200c'
]

In [11]:
Training_df_without_nationality['word_tokens_wrong'] = Training_df_without_nationality['word_tokens_wrong'].apply(lambda token_list: clean_list(token_list, symbols_to_remove))


In [12]:
Training_df_without_nationality

,word_tokens_wrong,error_index
0,"[افغانستان, , موضوع, :, اسراف, , اسراف, را, خد...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[افغانستان, , موضوع, :, اسراف, , اسراف, را, خد...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 0, 0..."
2,"[افغانستان, , موضوع, :, اسراف, , اسراف, را, خد...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[افغانستان, , موضوع, :, اسراف, , اسراف, را, خد...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 31, 31, 0, 0..."
4,"[افغانستان, , موضوع, :, اسراف, , اسراف, را, خد...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 0, 0..."
...,...,...
9195,"[ماه, بهار, تولد, مادرم, هست, ما, به, باغ, گلا...","[0, 0, 0, 0, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
9196,"[ماه, بهار, تولد, مادرم, هست, ما, به, باغ, گلا...","[0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0,..."
9197,"[ماه, بهار, تولد, مادرم, هست, ما, به, باغ, گلا...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0,..."
9198,"[ماه, بهار, تولد, مادرم, هست, ما, به, باغ, گلا...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
def is_list_of_zeros(lst):
    return isinstance(lst, list) and all(element == 0 for element in lst)

In [14]:
Training_df_without_nationality = Training_df_without_nationality[~Training_df_without_nationality['error_index'].apply(is_list_of_zeros)]

In [15]:
tdwn = Training_df_without_nationality

In [16]:
tdwn = tdwn.dropna(how='any')

In [ ]:
tdwn

,word_tokens_wrong,error_index
0,"[افغانستان, , موضوع, :, اسراف, , اسراف, را, خد...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[افغانستان, , موضوع, :, اسراف, , اسراف, را, خد...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 0, 0..."
2,"[افغانستان, , موضوع, :, اسراف, , اسراف, را, خد...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[افغانستان, , موضوع, :, اسراف, , اسراف, را, خد...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 31, 31, 0, 0..."
4,"[افغانستان, , موضوع, :, اسراف, , اسراف, را, خد...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 0, 0..."
...,...,...
9195,"[ماه, بهار, تولد, مادرم, هست, ما, به, باغ, گلا...","[0, 0, 0, 0, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
9196,"[ماه, بهار, تولد, مادرم, هست, ما, به, باغ, گلا...","[0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0,..."
9197,"[ماه, بهار, تولد, مادرم, هست, ما, به, باغ, گلا...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0,..."
9198,"[ماه, بهار, تولد, مادرم, هست, ما, به, باغ, گلا...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


**CREATING A VOCAB DICTIONARY**

In [17]:
all_tokens = [token for sublist in tdwn['word_tokens_wrong'] for token in sublist]

In [18]:
vocab = set(all_tokens)

In [19]:
vocab_list = list(vocab)

In [20]:
special_tokens = ['<sos>', '<eos>', '<pad>']
vocab_list = special_tokens + vocab_list

In [21]:
vocab_dict = {token: idx for idx, token in enumerate(vocab_list)}

# Optionally, you can add a special token for unknown words
vocab_dict['<unk>'] = len(vocab_dict)

**CONVERTING WORDES TO TOKENS USING DICTIONARY**

In [22]:
def words_to_indices_with_tokens(words, vocab_dict, sos_token='<sos>', eos_token='<eos>', unk_token='<unk>'):
    indices = [vocab_dict.get(sos_token)]  # Start with <sos>
    indices += [vocab_dict.get(word, vocab_dict.get(unk_token)) for word in words]  # Convert the words
    indices.append(vocab_dict.get(eos_token))  # End with <eos>
    return indices


In [23]:
tdwn['word_tokens_wrong'] = tdwn['word_tokens_wrong'].apply(lambda x: words_to_indices_with_tokens(x,vocab_dict))


**ADD PADDING TO TOKENISED INPUT SEQS**

In [24]:
max_len = tdwn['word_tokens_wrong'].apply(len).max()

In [25]:
# Define a function to pad the lists
def pad_sequence(sequence, max_len, pad_value):
    return sequence + [pad_value] * (max_len - len(sequence))

In [26]:
tdwn['word_tokens_wrong'] = tdwn['word_tokens_wrong'].apply(
    lambda x: pad_sequence(x, max_len, vocab_dict['<pad>'])
)

In [27]:
tdwn

,word_tokens_wrong,error_index
0,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 0, 0..."
2,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 31, 31, 0, 0..."
4,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 0, 0..."
...,...,...
9195,"[0, 1178, 3016, 5446, 7415, 4236, 8811, 5338, ...","[0, 0, 0, 0, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
9196,"[0, 1178, 3016, 5446, 7415, 4236, 8811, 5338, ...","[0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0,..."
9197,"[0, 1178, 3016, 5446, 7415, 4236, 8811, 5338, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0,..."
9198,"[0, 1178, 3016, 5446, 7415, 4236, 8811, 5338, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


**ADD PADDING TO ERROR INDEX SEQS**

In [28]:
def pad_sequence(sequence, max_length=209, pad_value=0):
    # Ensure the sequence is not longer than max_length
    sequence = sequence[:max_length]
    return sequence + [pad_value] * (max_length - len(sequence))


In [29]:
# Function to safely convert string to list
def safe_eval(value):
    import ast
    try:
        return ast.literal_eval(value)
    except:
        return []

# Convert string representation to actual list
tdwn['error_index'] = tdwn['error_index'].apply(safe_eval)



In [30]:
tdwn['error_index'] = tdwn['error_index'].apply(lambda x: pad_sequence(x, max_length=209, pad_value=-1))

In [31]:
tdwn

,word_tokens_wrong,error_index
0,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 0, 0..."
2,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 31, 31, 0, 0..."
4,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 0, 0..."
...,...,...
9195,"[0, 1178, 3016, 5446, 7415, 4236, 8811, 5338, ...","[0, 0, 0, 0, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
9196,"[0, 1178, 3016, 5446, 7415, 4236, 8811, 5338, ...","[0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0,..."
9197,"[0, 1178, 3016, 5446, 7415, 4236, 8811, 5338, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0,..."
9198,"[0, 1178, 3016, 5446, 7415, 4236, 8811, 5338, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


**ADD PADDING TO Wrong Word Tokens SEQS**

Attention Mask Creation

In [32]:
padding_value = 2
# Function to create attention mask
def create_attention_mask(sequence, padding_value):
    return [1 if token != padding_value else 0 for token in sequence]


In [34]:
tdwn['attention_mask'] = tdwn['word_tokens_wrong'].apply(lambda x: create_attention_mask(x, padding_value))

In [33]:
tdwn['error_index'] = tdwn['error_index'].apply(lambda lst: [1 if x!= 0 else x for x in lst])

In [35]:
# tdwn = tdwn.drop('list_length', axis=1)
# #

In [36]:
tdwn

,word_tokens_wrong,error_index,attention_mask
0,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[0, 1167, 3, 4284, 7767, 4875, 3, 4875, 2984, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
9195,"[0, 1178, 3016, 5446, 7415, 4236, 8811, 5338, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9196,"[0, 1178, 3016, 5446, 7415, 4236, 8811, 5338, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9197,"[0, 1178, 3016, 5446, 7415, 4236, 8811, 5338, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9198,"[0, 1178, 3016, 5446, 7415, 4236, 8811, 5338, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


**Creating embedding matrix**

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
# !unzip glove.6B.zip

from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

from gensim.models import KeyedVectors
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)


<ipython-input-39-8fbe5854946a>:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


In [ ]:
# Initialize the embedding matrix
embedding_dim = 100  # or the dimension of your word embeddings
vocab_size = len(vocab_dict)  # Size of your vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))
embedding_matrix.shape

(8847, 100)

In [ ]:
# if idx >= embedding_matrix.shape[0]:
#     print(f"Index {idx} is out of bounds. Valid range is 0 to {embedding_matrix.shape[0] - 1}")

In [ ]:
for word, idx in vocab_dict.items():
    if idx < embedding_matrix.shape[0]:
        if word in word2vec_model:
            embedding_matrix[idx] = word2vec_model[word]
        else:
            # Optional: initialize with random values if the word is not found
            embedding_matrix[idx] = np.random.normal(size=(embedding_dim,))
    else:
        print(f"Skipping out of bounds index: {idx}")


In [ ]:
# Fill the embedding matrix
for word, idx in vocab_dict.items():
    if word in word2vec_model:
        embedding_matrix[idx] = word2vec_model[word]
    else:
        embedding_matrix[idx] = np.random.normal(size=(embedding_dim,))  # Optional: random initialization


In [ ]:
# embedding_matrix.shape

In [ ]:
embedding_matrix_tensor = torch.tensor(embedding_matrix, dtype=torch.float)

**Create Label Tensors**

In [ ]:
label_mapping = {
    "True": 1,
    # "MainSigns_Consonants": 1,
    # "MainSigns_Vowels": 2,
    # "DiacriticSigns_Tashdid": 3,
    # "DiacriticSigns_Madd": 4,
    # "DiacriticSigns_Tanwin": 5,
    # "Form_Boundary": 6,
    # "Form_AlifLam": 7,
    # "Form_Place": 8,
    # "Form_Dot": 9,
    # "Punctuation": 10
    "False":0

}

# num_labels =11
num_labels =2

In [ ]:
# tdwn

In [37]:
data = {
    'input_ids': tdwn['word_tokens_wrong'],
    'attention_mask': tdwn['attention_mask'],
    'labels': tdwn['error_index'],
}

In [38]:
dataset = Dataset.from_dict(data)

In [39]:
# Shuffle the dataset
dataset = dataset.shuffle(seed=42)
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

train_test_split = train_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
valid_dataset = train_test_split['test']

In [41]:

# Save it as a CSV file
test_dataset.to_csv('test_dataset.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

3526676

In [ ]:
def format_labels(examples):
    return {'labels': [torch.tensor(label, dtype=torch.long) for label in examples['labels']]}

train_dataset = train_dataset.map(format_labels, batched=True)
test_dataset = test_dataset.map(format_labels, batched=True)

# Set the format to torch tensors
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/5888 [00:00<?, ? examples/s]

Map:   0%|          | 0/1840 [00:00<?, ? examples/s]

**Spiliting data set**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


**Adjust the Model**

In [ ]:
class CustomXLMRModel(XLMRobertaForTokenClassification):
    def __init__(self, config):
        super().__init__(config)
    def forward(self, input_ids=None, attention_mask=None, custom_embeddings=None, labels=None):
        if custom_embeddings is not None:
            outputs = self.roberta(
                inputs_embeds=custom_embeddings,
                attention_mask=attention_mask,
            )
        else:
            outputs = self.roberta(
                input_ids=input_ids,
                attention_mask=attention_mask,
            )

        logits = self.classifier(outputs[0])  # [batch_size, seq_length, num_labels]

        loss = None
        if labels is not None:
            # Flatten logits and labels for loss calculation
            logits = logits.view(-1, self.config.num_labels)  # [batch_size * seq_length, num_labels]
            labels = labels.view(-1)  # [batch_size * seq_length]

            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            print("Loss shape:", loss.shape)  # Should be an empty shape (scalar)

        return logits, loss

In [ ]:
# Set up model, config, embedding matrix, and trainer
config = XLMRobertaConfig.from_pretrained('xlm-roberta-base')
config.hidden_size = 100  # Example for your embedding matrix with dim 100
config.num_attention_heads = 10  # Adjust to match the embedding size (100 / 10 = 10)

model = CustomXLMRModel(config)
model.to(device)
model.roberta.embeddings.word_embeddings = torch.nn.Embedding.from_pretrained(embedding_matrix_tensor)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,                # Log every 10 steps
    evaluation_strategy="steps",     # Evaluation strategy ("steps" or "epoch")
    save_steps=500,                  # Save checkpoint every 500 steps
    save_total_limit=3,              # Limit the total amount of checkpoints
    learning_rate=2e-5,              # Learning rate
)
# Assuming you have training_args, train_dataset, and test_dataset already defined:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=test_dataset,

)




/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


**PREPARING MODEL**

**TRAIN MODEL**

In [ ]:
# Start training
# trainer.train()

In [ ]:
# # saving model
# save_directory = './drive/MyDrive/Error_detection /XLMR/'
# model.save_pretrained(save_directory)


In [ ]:
# torch.cuda.empty_cache()

In [ ]:
# text = ['خداحافظی','عزیزم']

# # Tokenize the input
# inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
# inputs = {key: value.to(device) for key, value in inputs.items()}
# # print("Token IDs:", inputs["input_ids"])
# # print("Attention Mask:", inputs["attention_mask"])
# if "token_type_ids" in inputs:
#     print("Token Type IDs:", inputs["token_type_ids"])
# list_2=[]
# for token in inputs["input_ids"][0]:
#   decoded_text = tokenizer.decode(token, skip_special_tokens=True)
#   list_2.append(decoded_text)
#   print(token,"   :  ",decoded_text)

# # print(list_2)
# # Ensure the model is in evaluation mode
# model.eval()

# # Perform the prediction
# with torch.no_grad():
#     outputs = model(**inputs)


# logits = outputs.logits
# probs = torch.nn.functional.softmax(logits, dim=-1)
# probs=probs.cpu()
# numpy_array = probs.numpy()
# final_output=[]
# # Convert NumPy array to Python list
# for i in numpy_array[0]:
#   values=i.tolist()
#   index = values.index(max(values))

#     # Convert the maximum value to the nearest integer
#   nearest_int = int(index)
#   final_output.append(index)
# print(final_output)